In [ ]:
import torch
from tqdm import tqdm
import numpy as np
import torch.nn as nn
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class HangmanPlayer:
    def __init__(self, word, model, lives=6):
        self.original_word = word
        self.full_word = [ord(i)-97 for i in word]
        self.letters_guessed = set([])
        self.letters_remaining = set(self.full_word)
        self.lives_left = lives
        self.obscured_words_seen = []
        self.letters_previously_guessed = []
        self.guesses = []
        self.correct_responses = []
        self.z = model
        return

    def encode_obscured_word(self):
        obscured_word = np.zeros((len(self.full_word), 29), dtype=np.float32)
        guessed_mask = np.array([i in self.letters_guessed for i in self.full_word], dtype=np.bool_)
        obscured_word[np.arange(len(self.full_word)), np.where(guessed_mask, self.full_word, 26)] = 1
        return obscured_word

    def encode_guess(self, guess):
        encoded_guess = np.zeros(26, dtype=np.float32)
        encoded_guess[guess] = 1
        return(encoded_guess)

    def encode_previous_guesses(self):
        guess = np.zeros(26, dtype=np.float32)
        for i in self.letters_guessed:
            guess[i] = 1
        return(guess)

    def encode_correct_responses(self):
        response = np.zeros(26, dtype=np.float32)
        for i in self.letters_remaining:
            response[i] = 1.0
        response /= response.sum()
        return(response)

    def store_guess_and_result(self, guess):
        self.obscured_words_seen.append(self.encode_obscured_word())
        self.letters_previously_guessed.append(self.encode_previous_guesses())

        self.guesses.append(guess)
        self.letters_guessed.add(guess)

        correct_responses = self.encode_correct_responses()
        self.correct_responses.append(correct_responses)

        if guess in self.letters_remaining:
            self.letters_remaining.remove(guess)

        if self.correct_responses[-1][guess] < 0.00001:
            self.lives_left -= 1
        return

    def run(self):
        while self.lives_left > 0 and len(self.letters_remaining) > 0:
            obscured_word_tensor = torch.tensor(self.encode_obscured_word(), dtype=torch.float32).to(device)
            previous_guesses_tensor = torch.tensor(self.encode_previous_guesses(), dtype=torch.float32).to(device)

            obscured_word_tensor = obscured_word_tensor.unsqueeze(0)
            previous_guesses_tensor = previous_guesses_tensor.unsqueeze(0)

            with torch.no_grad():
                outputs = self.z(obscured_word_tensor, previous_guesses_tensor)
                guess = torch.argmax(outputs, dim=1).item()

            self.store_guess_and_result(guess)

        return (np.array(self.obscured_words_seen),
                np.array(self.letters_previously_guessed),
                np.array(self.correct_responses))

    def show_words_seen(self):
        for word in self.obscured_words_seen:
            print(''.join([chr(i + 97) if i != 26 else ' ' for i in word.argmax(axis=1)]))

    def show_guesses(self):
        for guess in self.guesses:
            print(chr(guess + 97))

    def play_by_play(self):
        print('Hidden word was "{}"'.format(self.original_word))
        for i in range(len(self.guesses)):
            word_seen = ''.join([chr(i + 97) if i != 26 else ' ' for i in self.obscured_words_seen[i].argmax(axis=1)])
            print('Guessed {} after seeing "{}"'.format(chr(self.guesses[i] + 97),
                                                        word_seen))

    def evaluate_performance(self):
        ended_in_success = self.lives_left > 0
        letters_in_word = set([i for i in self.original_word])
        correct_guesses = len(letters_in_word) - len(self.letters_remaining)
        incorrect_guesses = len(self.guesses) - correct_guesses
        return(ended_in_success, correct_guesses, incorrect_guesses, letters_in_word)

In [ ]:
with open('/content/words_250000_train.txt', 'r') as f:
    words = [line.strip() for line in f.readlines()]

np.random.seed(42)
np.random.shuffle(words)

train_val_split_idx = int(len(words) * 0.8)
print('Training with {} WordNet words'.format(train_val_split_idx))

MAX_NUM_INPUTS = max([len(i) for i in words[:train_val_split_idx]])
EPOCH_SIZE = train_val_split_idx
NUM_EPOCHS = 3
NUM_CLASSES = 26
BATCH_SIZE = np.array([len(i) for i in words[:train_val_split_idx]]).mean()
print('Max word length: {}, average word length: {:0.1f}'.format(MAX_NUM_INPUTS, BATCH_SIZE))

In [ ]:
class LSTMNet(nn.Module):
    def __init__(self):
        super(LSTMNet, self).__init__()
        self.lstm = nn.LSTM(input_size=MAX_NUM_INPUTS, hidden_size=128, batch_first=True)
        self.dense = nn.Linear(128 + 26, NUM_CLASSES)

    def forward(self, input_obscured_word_seen, input_letters_guessed_previously):
        lstm_out, _ = self.lstm(input_obscured_word_seen)
        final_lstm_output = lstm_out[:, -1, :]
        combined_input = torch.cat((final_lstm_output, input_letters_guessed_previously), dim=1)
        output = self.dense(combined_input)
        return output

model = LSTMNet().to(device)

BATCH_SIZE = 128
sequence_length = 10
learning_rate = 0.001
step_size = 1
gamma = 0.92

input_obscured_word_seen = torch.randn(BATCH_SIZE, 10, MAX_NUM_INPUTS).to(device)
input_letters_guessed_previously = torch.randn(BATCH_SIZE, 26).to(device)

output = model(input_obscured_word_seen, input_letters_guessed_previously)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

def train_model(data_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct_predictions = 0

        with tqdm(total=len(data_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
            for inputs, correct_responses in data_loader:
                inputs = inputs.to(device)
                correct_responses = correct_responses.to(device)

                optimizer.zero_grad()

                outputs = model(*inputs)
                loss = loss_function(outputs, correct_responses)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == correct_responses).sum().item()

                pbar.set_postfix(loss=loss.item())
                pbar.update(1)

        avg_loss = total_loss / len(data_loader)
        classification_error = 1 - (correct_predictions / len(data_loader.dataset))

        print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}, Classification Error: {classification_error:.4f}')
        scheduler.step()

In [ ]:
total_samples = 0

for epoch in range(NUM_EPOCHS):
    model.train()
    i = 0
    progress_bar = tqdm(total=len(words), desc=f'Epoch {epoch + 1}/{NUM_EPOCHS}', unit='sample')

    while total_samples < (epoch + 1) * EPOCH_SIZE:
        if i >= len(words):
            break

        word = words[i]
        i += 1

        other_player = HangmanPlayer(word, model)
        words_seen, previous_letters, correct_responses = other_player.run()

        words_seen = torch.tensor(words_seen, dtype=torch.float32).to(device)
        previous_letters = torch.tensor(previous_letters, dtype=torch.float32).to(device)
        correct_responses = torch.tensor(correct_responses, dtype=torch.float32).to(device)

        inputs = (words_seen, previous_letters)

        optimizer.zero_grad()
        outputs = model(*inputs)
        loss = loss_function(outputs, correct_responses)
        loss.backward()
        optimizer.step()

        total_samples += 1
        progress_bar.update(1)

        progress_bar.set_postfix(loss=loss.item())

    progress_bar.close()
    print(f'Epoch {epoch + 1} completed. Total Samples: {total_samples}')

In [ ]:
torch.save(model.state_dict(), 'lstm_model.pth')
print('Model saved to lstm_model.pth')

In [ ]:
import pandas as pd

In [ ]:
def evaluate_model(my_words, my_model):
    results = []

    for word in tqdm(my_words, desc="Evaluating Words"):
        my_player = HangmanPlayer(word, my_model)
        _ = my_player.run()
        results.append(my_player.evaluate_performance())

    df = pd.DataFrame(results, columns=['won', 'num_correct', 'num_incorrect', 'letters'])
    return df

result_df = evaluate_model(words[train_val_split_idx:], model)

print('Performance on the validation set:')
print('- Averaged {:0.1f} correct and {:0.1f} incorrect guesses per game'.format(result_df['num_correct'].mean(),
                                                                       result_df['num_incorrect'].mean()))
print('- Won {:0.1f}% of games played'.format(100 * result_df['won'].sum() / len(result_df.index)))